In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import seaborn as sns 
import matplotlib.pyplot as plt
import random
import tensorflow as tf

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
tf.random.set_seed(42)

In [ ]:
train = pd.read_csv(r'/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv(r'/kaggle/input/digit-recognizer/test.csv')
y_train = train['label']
X_train = train.drop('label', axis=1)
y_train.head()

In [ ]:
X_train.head()

In [ ]:
X_train.info()

In [ ]:
X_train = X_train/255
y_train.nunique()

In [ ]:
sns.countplot(x=y_train)
plt.title('# Of samples', size=14)
plt.show()

In [ ]:
fig, ax = plt.subplots(5, 5, figsize=(8, 8))
fig.suptitle('Digits images and labels', fontsize=16)
ax = ax.ravel()
for i in range(25):
    sample_n = random.randint(0, X_train.shape[0])
    ax[i].imshow(np.array(X_train.iloc[sample_n]).reshape(28, 28), cmap='vlag')
    ax[i].get_xaxis().set_visible(False)
    ax[i].get_yaxis().set_visible(False)
    ax[i].set_title(y_train[sample_n], fontsize = 12)

plt.subplots_adjust(hspace=0.3)

In [ ]:
plt.imshow(np.array(X_train.mean()).reshape(28, 28), cmap='inferno')
plt.colorbar()
plt.title('average shape', {'fontsize': 16})
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(8, 4))
fig.suptitle('Average shape per digit', fontsize=16)

ax = ax.ravel()

for i in range(10):
    ax[i].imshow(np.array(train[train['label'] == i].drop('label', axis=1).mean()).reshape(28, 28), cmap='inferno')
    ax[i].get_xaxis().set_visible(False)
    ax[i].get_yaxis().set_visible(False)
    ax[i].set_title(i, fontsize = 12)

In [ ]:
X_train_array = np.array(X_train)
X_train_array

In [ ]:
test_array = np.array(test)
test_array

In [ ]:
X_train_array.shape

In [ ]:
test_array.shape

In [ ]:
X_train_array = X_train_array.reshape(42000, 28, 28, 1)
X_train_array.shape

In [ ]:
test_array = test_array.reshape(-1, 28, 28, 1)
test_array.shape

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="constant",
    
    horizontal_flip=False,
    vertical_flip=False,
    
    rotation_range=20,
    
    validation_split=0.2
)

In [ ]:
datagen.fit(X_train_array)

In [ ]:
tf.random.set_seed(42)

In [ ]:
'''
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(48, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=784, activation='relu', input_shape=(784, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=392, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=151, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=50, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=10, activation='softmax'),
]) '''

In [ ]:
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Dense, Input, MaxPooling2D, Flatten, BatchNormalization, Concatenate, Reshape,Dropout

In [ ]:
from keras.utils import np_utils

In [ ]:
y_train = np_utils.to_categorical(y_train)

In [ ]:
y_train

In [ ]:
input_layer = Input((28,28,1))
conv2d_1_1 = Conv2D(filters=128, kernel_size=[2,2], activation="relu", padding="same", name ="conv2d_branch1")(input_layer)
b_n_1 = BatchNormalization()(conv2d_1_1)
maxpool = MaxPooling2D(pool_size=[2,2])(b_n_1)
drop = Dropout(0.2)(maxpool)

conv2d_2_2 = Conv2D(filters=256, kernel_size=[2,2], activation="relu", padding="same")(drop)
b_n_2 = BatchNormalization()(conv2d_2_2)
maxpool = MaxPooling2D(pool_size=[2,2])(b_n_2)
drop = Dropout(0.2)(maxpool)

conv2d_3_3 = Conv2D(filters=512, kernel_size=[2,2], activation="relu", padding="same")(drop)
b_n_2_3 = BatchNormalization()(conv2d_3_3)
maxpool = MaxPooling2D(pool_size=[2,2])(b_n_2_3)
drop = Dropout(0.2)(maxpool)

conv2d_3_4 = Conv2D(filters=720, kernel_size=[2,2], activation="relu", padding="same")(drop)
flatten_c2d_3 = Flatten()(conv2d_3_4)
conv_dense_1 = Dense(512, activation="relu")(flatten_c2d_3)
bn = BatchNormalization()(conv_dense_1)
drop = Dropout(0.2)(bn)
conv_dense_1 = Dense(256, activation="relu")(drop)
bn = BatchNormalization()(conv_dense_1)
drop_dense_last = Dropout(0.2)(bn)

conv2d_1 = Conv2D(filters=128, kernel_size=[2,2], activation="relu",dilation_rate = 3, padding="same", name ="conv2d_branch2")(input_layer)
b_n = BatchNormalization()(conv2d_1)
maxpool = MaxPooling2D(pool_size=[2,2])(b_n)
drop = Dropout(0.2)(maxpool)

conv2d_2 = Conv2D(filters=256, kernel_size=[2,2], activation="relu",dilation_rate = 3, padding="same")(drop)
b_n_2 = BatchNormalization()(conv2d_2)
maxpool = MaxPooling2D(pool_size=[2,2])(b_n_2)
drop = Dropout(0.2)(maxpool)

conv2d_3 = Conv2D(filters=512, kernel_size=[2,2], activation="relu",dilation_rate = 3, padding="same")(drop)
b_n_2 = BatchNormalization()(conv2d_3)
maxpool = MaxPooling2D(pool_size=[2,2])(b_n_2)
drop = Dropout(0.2)(maxpool)

conv2d_3 = Conv2D(filters=720, kernel_size=[2,2], activation="relu", padding="same")(drop)
flatten_c2d = Flatten()(conv2d_3)
conv_dense_2 = Dense(512, activation="relu")(flatten_c2d)
bn = BatchNormalization()(conv_dense_2)
drop = Dropout(0.2)(bn)
conv_dense_2 = Dense(256, activation="relu")(drop)
bn = BatchNormalization()(conv_dense_2)
drop_dense_last_2 = Dropout(0.2)(bn)

concat = Concatenate()([drop_dense_last, drop_dense_last_2])
# decode_dense = Dense(256, activation="relu")(drop)
# drop = Dropout(0.2)(decode_dense)
# decode_dense_2 = Dense(256, activation="relu")(concat)
# drop = Dropout(0.27)(decode_dense_2)
# decode_dense_2 = Dense(128, activation="relu")(drop)
# drop = Dropout(0.27)(decode_dense_2)
decode_dense_2 = Dense(256, activation="relu")(concat)
bn = BatchNormalization()(decode_dense_2)
drop = Dropout(0.2)(bn)

decode_dense_2 = Dense(128, activation="relu")(drop)
bn = BatchNormalization()(decode_dense_2)
drop = Dropout(0.2)(bn)

decode_dense_2 = Dense(128, activation="relu")(drop)
bn = BatchNormalization()(decode_dense_2)
drop = Dropout(0.2)(bn)

output_dense = Dense(10, activation="softmax")(drop)
model = Model(inputs=[input_layer], outputs=[output_dense])
model.compile(
    optimizer=Adam(lr=1e-3), loss="categorical_crossentropy", metrics=["accuracy"]
)
print(model.summary())
tf.keras.utils.plot_model(model, to_file='model_plot2.png', show_shapes=False, show_layer_names=False)

In [ ]:
history_new = model.fit(datagen.flow(X_train_array, y_train, batch_size=250, subset='training'),
                    epochs=250,
                    validation_data=datagen.flow(X_train_array, y_train,
         batch_size=10, subset='validation'))

In [ ]:
'''
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.008,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)'''

In [ ]:
'''history = model.fit(datagen.flow(X_train_array, y_train, batch_size=50, subset='training'),
                    epochs=100,
                    validation_data=datagen.flow(X_train_array, y_train,
         batch_size=10, subset='validation'),
                    callbacks=[es])

In [ ]:
'''model.fit(X_train_array,
        y_train,
        batch_size=50,
        epochs=100,
        validation_split=0.3,
        callbacks=[es])

In [ ]:
plt.figure(figsize = (12,10))
plt.title('Model loss', fontsize = 14)
plt.plot(range(len(history_new.history['loss'])), history_new.history['loss'], marker='o', c='gray')
plt.plot(range(len(history_new.history['loss'])), history_new.history['val_loss'], marker='o')
plt.legend(labels=['training loss', 'validation loss'])
plt.show()

In [ ]:
plt.figure(figsize = (12,10))
plt.title('Model accuracy', fontsize = 14)
plt.plot(range(len(history_new.history['loss'])), history_new.history['accuracy'], c='gray', marker='o')
plt.plot(range(len(history_new.history['loss'])), history_new.history['val_accuracy'], marker='o')
plt.legend(labels=['training accuracy', 'validation accuracy'])
plt.show()

In [ ]:
test_array = test_array / 255
test_result = model.predict(test_array)
test_result = np.array(pd.DataFrame(test_result).idxmax(axis=1))
fig, ax = plt.subplots(5, 5, figsize=(8, 8))
fig.suptitle('Digits images and labels', fontsize=16)
ax = ax.ravel()
for i in range(25):
    sample_n = random.randint(0, test.shape[0])
    ax[i].imshow(np.array(test.iloc[sample_n]).reshape(28, 28), cmap='inferno')
    ax[i].get_xaxis().set_visible(False)
    ax[i].get_yaxis().set_visible(False)
    ax[i].set_title(test_result[sample_n], fontsize = 12)

plt.subplots_adjust(hspace=0.3)
fig.show()

In [ ]:
ev = model.evaluate(X_train_array, y_train)
ev

In [ ]:
pred = model.predict(X_train_array)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
#y_train.unique()
y_train=np.argmax(y_train, axis=1)

In [ ]:
y_train

In [ ]:
pred = np.array(pd.DataFrame(pred).idxmax(axis=1))

In [ ]:
print(classification_report(y_train, pred))

In [ ]:
plt.figure(figsize=(12,10))
plt.title('Predicted digits', size=14)
sns.heatmap(confusion_matrix(y_train, pred), cmap='inferno', annot=True, fmt = '.0f')
plt.show()

In [ ]:
predicted_data = pd.DataFrame({
    'ImageId': test.index+1,
    'Label': test_result
})

In [ ]:
predicted_data.to_csv('pred.csv', index=False)